# Data transformation

In [1]:
# Ignore warnings :
import warnings
warnings.filterwarnings('ignore')

# Data handling
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import itertools
import numpy as np

In [2]:
diamonds=pd.read_csv('/home/carmencuadrado/Ironhack/ih_datamadpt0420_project_m2/diamonds_supercleaned.csv')

## Categorizing numerical variables

In order to make diamonds dataframe more actionable we categorize 

- **Table** into 
    - Excellent, if it takes values between 53 and 58
    - Very good, if it takes values between 52 adn 53 or between 58 and 60
    - Upper outlier, if it takes values over 60
    - Lower outlier, if it takes values under 52
    
    For this sake, we create the *table_type* column
    
    
- **Depth** into
    - Excellent, if it takes values between 59 and 62.3
    - Very good, if it takes values between 58 adn 58.9 or between 62.4 and 63.8
    - Upper outlier, if it takes values over 63.8
    - Lower outlier, if it takes values under 59
    
    For this sake, we create the *depth_type* column


- **Carat** into scales creating the categorical type column *carat_scale*

In [3]:
diamonds['table_type']=diamonds['table'].apply(lambda x: "Excellent" if x>=53 and x<=58 else ("Very good" if x>=52 and x<=53 else ("Very good" if x>=58 and x<=60 else ("Upper outlier" if x>60 else "Lower outlier"))))
diamonds['carat_scale']=diamonds['carat'].apply(lambda x: "0-0.10" if x>0 and x<=0.10 else("0.10-0.25" if x>0.10 and x<=0.25 else ("0.25-0.50" if x>0.25 and x<=0.50 else ("0.50-0.75" if x>0.50 and x<=0.75 else ("0.75-1" if x>0.75 and x<=1 else ("1-1.25" if x >1 and x<=1.25 else ("1.25-1.50" if x>1.25 and x<=1.50 else ("1.50-1.75" if x>1.50 and x<=1.75 else ("1.75-2" if x>1.75 and x<=2 else ("2-2.5" if x>2 and x<=2.5 else ("2.5-3" if x>2.5 and x<3 else ("3-4" if x>3 and x<=4 else "+4"))))))))))))
diamonds['depth_type']=diamonds['depth'].apply(lambda x: "Excellent" if x>=59 and x<=62.3 else ("Very good" if x>=58 and x<=58.9 else ("Very good" if x>=62.4 and x<=63.8 else ("Upper outlier" if x>63.8 else "Lower outlier"))))

In [4]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,table_type,carat_scale,depth_type
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25,Excellent,1-1.25,Very good
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75,Excellent,0.25-0.50,Very good
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65,Excellent,0.50-0.75,Upper outlier
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00,Excellent,0.25-0.50,Very good
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95,Very good,1-1.25,Excellent


## Enriching the dataframe

#### Attributing a cut to each diamond

In [83]:
url='https://www.diamonds.pro/guides/diamond-proportion/'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')
diamonds_proportions = soup.find_all('td')
x=diamonds_proportions[10:]

In [84]:
split = 6
x = [x[i:i + split] for i in range(0, len(x), split)]

lista=[]
for i in range(0, len(x)):
    lista.append(x[i][0])
    lista.append(x[i][1])
    lista.append(x[i][2])
    lista.append(x[i][5])
    
lista2=[]
for i in lista:
    i=re.sub('\<.*?>','',str(i))
    x=re.sub('–\xa0','',str(i))
    x=re.sub('%','',str(i))
    lista2.append(x)
    
lista3=[]
s=4
lista3=[lista2[i:i + s] for i in range(0,len(lista2),s)]

lista4=[]
for i in range(0,len(lista3)):
    lista4.append(lista3[i][0])
    x=re.split('–|;',str(lista3[i][1]))
    for a in x:
        lista4.append(a)
    y=re.split('–|;',str(lista3[i][2]))
    for b in y:
        lista4.append(b)
    z=re.split('–|;',str(lista3[i][3]))
    for c in z:
        lista4.append(c)
        
lista4

['Round Brilliant Cut',
 '59 ',
 ' 62.6',
 '54 ',
 ' 57',
 '1.0 ',
 ' 1.03',
 'Princess Cut',
 '68 ',
 ' 74',
 '69 ',
 ' 75',
 '1.0 ',
 ' 1.04',
 'Cushion Cut',
 '61 ',
 '\xa0 68',
 '&lt',
 ' 68',
 '1.0 ',
 ' 1.08 (sq)1.15 ',
 ' 1.25 (rect)',
 'Emerald Cut',
 '61 ',
 ' 68',
 '61 ',
 ' 69',
 '1.30 ',
 ' 1.45',
 'Asscher Cut',
 '61 ',
 ' 68',
 '61 ',
 ' 69',
 '1.0 ',
 ' 1.05',
 'Oval Cut',
 '&lt',
 ' 63',
 '53 ',
 '\xa0 63',
 '1.30 ',
 ' 1.50',
 'Pear Shape',
 '&lt',
 ' 68',
 '53 ',
 '\xa0 65',
 '1.45 ',
 ' 1.75',
 'Radiant Cut',
 '&lt',
 ' 67',
 '61 ',
 '\xa0 69',
 '1.0 ',
 ' 1.05 (sq)1.20 ',
 ' 1.50 (rect)',
 'Heart Shape',
 '56 ',
 ' 66',
 '56 ',
 ' 62',
 '1.0 (or as close as possible)',
 'Marquise Cut',
 '58 ',
 ' 62',
 '53 to 63',
 '1.85 ',
 ' 2.1']

In [79]:
sp=7
lista4=[lista4[i:i + sp] for i in range(0,len(lista4),sp)]

[['Round Brilliant Cut', '59 ', ' 62.6%', '54 ', ' 57%', '1.0 ', ' 1.03'],
 ['Princess Cut', '68 ', ' 74%', '69 ', ' 75%', '1.0 ', ' 1.04'],
 ['Cushion Cut',
  '61  68%',
  '&lt',
  ' 68%',
  '1.0 ',
  ' 1.08 (sq)1.15 ',
  ' 1.25 (rect)'],
 ['Emerald Cut', '61 ', ' 68%', '61 ', ' 69%', '1.30 ', ' 1.45'],
 ['Asscher Cut', '61 ', ' 68%', '61 ', ' 69%', '1.0 ', ' 1.05'],
 ['Oval Cut', '&lt', ' 63%', '53  63%', '1.30 ', ' 1.50', 'Pear Shape'],
 ['&lt', ' 68%', '53  65%', '1.45 ', ' 1.75', 'Radiant Cut', '&lt'],
 [' 67%',
  '61  69%',
  '1.0 ',
  ' 1.05 (sq)1.20 ',
  ' 1.50 (rect)',
  'Heart Shape',
  '56 '],
 [' 66%',
  '56 ',
  ' 62%',
  '1.0 (or as close as possible)',
  'Marquise Cut',
  '58 ',
  ' 62%'],
 ['53 to 63%', '1.85 ', ' 2.1']]

In [9]:
diamonds['L/W ration']=diamonds['x']/diamonds['y']

In [ ]:
datos = diamonds
d=['D','E','F','G','H','I']
b=['VS2', 'VS1', 'IF', 'VVS1', 'VVS2']
e=['Excellent', 'Very good']

D=['D','E','F']
B=['IF', 'VVS1', 'VVS2']

dd=['D','E','F','G','H']
bb=['IF', 'VVS1']

condiciones = [diamonds.color.isin(D) & diamonds.clarity.isin(B) & diamonds.table_type.isin(e),
               diamonds.color.isin(dd) & diamonds.clarity.isin(bb) & diamonds.table_type.isin(e),diamonds.color.isin(d) & diamonds.clarity.isin(b) & diamonds.table_type.isin(e)]
elecciones = ['Piaget', 'Cartier','Tiffanys']
diamonds["Potential Client"] = np.select(condiciones, elecciones, default="Common")

#### Matching each diamonds with a potential client

In [ ]:
diamonds['diamonds_type']=np.where(diamonds['Round length']==diamonds['x_x'], 'Round', diamonds['diamonds_type'])

In [ ]:
datos = diamonds
d=['D','E','F','G','H','I']
b=['VS2', 'VS1', 'IF', 'VVS1', 'VVS2']
e=['Excellent', 'Very good']

D=['D','E','F']
B=['IF', 'VVS1', 'VVS2']

dd=['D','E','F','G','H']
bb=['IF', 'VVS1']

condiciones = [diamonds.color.isin(D) & diamonds.clarity.isin(B) & diamonds.table_type.isin(e),
               diamonds.color.isin(dd) & diamonds.clarity.isin(bb) & diamonds.table_type.isin(e),diamonds.color.isin(d) & diamonds.clarity.isin(b) & diamonds.table_type.isin(e)]
elecciones = ['Piaget', 'Cartier','Tiffanys']
diamonds["Potential Client"] = np.select(condiciones, elecciones, default="Common")

In [ ]:
diamonds["Potential Client"].unique()

## Ranking diamonds

Sorting by numerical variables

In [5]:
diamonds['table_type'] = pd.Categorical(diamonds['table_type'], ['Lower outlier','Upper outlier','Very good','Excellent'])
diamonds['carat_scale'] = pd.Categorical(diamonds['carat_scale'], ["0-0.10","0.10-0.25","0.25-0.50","0.50-0.75","0.75-1","1-1.25","1.25-1.50","1.50-1.75","1.75-2","2-2.5","2.5-3","3-4","+4"])
diamonds['depth_type'] = pd.Categorical(diamonds['depth_type'], ['Lower outlier','Upper outlier','Very good','Excellent'])

In [6]:
diamonds.sort_values(['table_type','carat_scale','depth_type'],ascending=(False,False,False))
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,table_type,carat_scale,depth_type
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25,Excellent,1-1.25,Very good
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75,Excellent,0.25-0.50,Very good
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65,Excellent,0.50-0.75,Upper outlier
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00,Excellent,0.25-0.50,Very good
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95,Very good,1-1.25,Excellent


Sorting by categorical variables

In [7]:
diamonds['cut'] = pd.Categorical(diamonds['cut'], ["Fair", "Good","Very Good","Premium","Ideal"])
diamonds['clarity'] = pd.Categorical(diamonds['clarity'], ["I1", "I2","I3","SI1","SI2","VS1","VS2","VVS1","VVS2","IF"])
diamonds['color'] = pd.Categorical(diamonds['color'], ["I","J","H","G","F","E","D"])

In [8]:
diamonds.sort_values('price').head()

,carat,cut,color,clarity,depth,table,price,x,y,z,table_type,carat_scale,depth_type
18377,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,Upper outlier,0.10-0.25,Excellent
9782,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,Excellent,0.10-0.25,Excellent
18381,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,Upper outlier,0.10-0.25,Lower outlier
20968,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,Excellent,0.25-0.50,Very good
12084,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,Excellent,0.25-0.50,Very good


Sorting by numerical & categorical variables

In [9]:
diamonds.sort_values(['cut','clarity','color','table_type','carat_scale','depth_type'],ascending=(False,False,False,False,False,False)).reset_index()
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,table_type,carat_scale,depth_type
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25,Excellent,1-1.25,Very good
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75,Excellent,0.25-0.50,Very good
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65,Excellent,0.50-0.75,Upper outlier
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00,Excellent,0.25-0.50,Very good
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95,Very good,1-1.25,Excellent


In [34]:
round=pd.DataFrame()
x=d['Round']
width=[]
carat=[]
for i in x:
    width.append(i[0])
    carat.append(i[1])
round['Round length']=width
round['carat']=carat
round['Round length']=round['Round length'].apply(clean)
round['carat']=round['carat'].apply(clean)
round['Round length']=round['Round length'].apply(clean2)
round['carat']=round['carat'].apply(clean1)
round['Round length']=round['Round length'].astype('float64', copy=False)
round['carat']=round['carat'].astype('float64', copy=False)
diamonds= pd.merge(diamonds,round, on='carat', how='left')
diamonds['Round length']=diamonds['Round length'].fillna(0)


In [58]:
diamonds.to_csv(r'/home/carmencuadrado/Ironhack/ih_datamadpt0420_project_m2/diamonds_result.csv')

In [63]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')